In [1]:
import os
os.chdir('../')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
import scanpy as sc
import torch
import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt
import numpy as np
import gdown
import anndata as ad
import pandas as pd

 captum (see https://github.com/pytorch/captum).
INFO:lightning_fabric.utilities.seed:Global seed set to 0
/home/alextopalova/miniconda3/envs/archmap/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(12, 3))
torch.set_printoptions(precision=3, sci_mode=False, edgeitems=7)

In [4]:
condition_key = 'batch' 
cell_type_key = 'final_annotation'
target_conditions = ['10X']

surgery_path = 'surgery_model_proper'
output = 'Immune_ALL_human.h5ad'

In [5]:
adata_all = sc.read(output)

source_adata = adata_all[~adata_all.obs[condition_key].isin(target_conditions)].copy()
target_adata = adata_all[adata_all.obs[condition_key].isin(target_conditions)].copy()

In [6]:
model = sca.models.TRVAE.load(surgery_path, adata_all)

AnnData object with n_obs × n_vars = 33506 × 12303
    obs: 'batch', 'chemistry', 'data_type', 'dpt_pseudotime', 'final_annotation', 'mt_frac', 'n_counts', 'n_genes', 'sample_ID', 'size_factors', 'species', 'study', 'tissue'
    layers: 'counts'

INITIALIZING NEW NETWORK..............
Encoder Architecture:
	Input Layer in, out and cond: 12303 128 10
	Hidden Layer 1 in/out: 128 128
	Mean/Var Layer in/out: 128 10
Decoder Architecture:
	First Layer in, out and cond:  10 128 10
	Hidden Layer 1 in/out: 128 128
	Output Layer in/out:  128 12303 



In [7]:
full_latent = sc.AnnData(model.get_latent(adata_all.X, adata_all.obs[condition_key]))
full_latent.obs['cell_type'] = adata_all.obs[cell_type_key].tolist()
full_latent.obs['batch'] = adata_all.obs[condition_key].tolist()

In [8]:
os.chdir("./uncert")
from uncert_metric import integration_uncertain

In [9]:
result = integration_uncertain(full_latent, condition_key)

In [10]:
result

,batch,uncertainty
0,10X,0.757693
1,Freytag,0.307533
2,Oetjen_A,0.374256
3,Oetjen_P,0.540864
4,Oetjen_U,0.345338
5,Sun_sample1_CS,0.286294
6,Sun_sample2_KC,0.284884
7,Sun_sample3_TB,0.283747
8,Sun_sample4_TC,0.291773
9,Villani,0.469243
